 Скрипт для обучения и сохранения модели классификации изображений
 с использованием библиотек TensorFlow и Keras.

 Описание:
 Этот скрипт предназначен для создания и обучения модели машинного обучения,
 способной классифицировать изображения фруктов и овощей. Модель основана на 
 предварительно обученной сети MobileNetV2, что позволяет добиться хорошей 
 точности классификации. Обученная модель сохраняется в файл 'my_classification_model.h5'
 для дальнейшего использования в других приложениях.

 Используемые библиотеки:
 - TensorFlow и Keras для создания и обучения модели.
 - ImageDataGenerator для аугментации и обработки изображений.
 - MobileNetV2 предоставляет предварительно обученную базовую сеть.
 - Другие библиотеки, такие как os, shutil и random, для управления данными и файлами.

Шаги выполнения:
1. Устанавливаются пути к папкам с данными (набором изображений фруктов и овощей).
2. Задаются параметры обработки изображений, такие как размер изображений и аугментации.
3. Создаются генераторы для данных обучения и валидации.
4. Создается и обучается модель. Сначала загружается предварительно обученная сеть MobileNetV2,
    затем к ней добавляются дополнительные слои для классификации.
5. Модель компилируется с оптимизатором 'adam' и функцией потерь 'categorical_crossentropy'.
6. Обучение модели выполняется в течение нескольких эпох с использованием генераторов данных.
7. Обученная модель сохраняется в файл 'my_classification_model.h5' для будущего использования.



In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import shutil
import random

Выводим названия папок

In [4]:
def list_folders(directory):
    # Проверяем, что директория существует
    if not os.path.exists(directory):
        print(f"Директория '{directory}' не существует.")
        return

    # Получаем все элементы в директории
    items = os.listdir(directory)

    # Фильтруем только папки
    folders = [item for item in items if os.path.isdir(os.path.join(directory, item))]

    # Выводим названия папок
    print("Названия папок:")
    for folder in folders:
        print(folder)

# Указываем путь к директории
directory_path = './train'

# Выводим названия папок в заданной директории
list_folders(directory_path)

Названия папок:
apple
banana
beetroot
bell pepper
cabbage
capsicum
carrot
cauliflower
chilli pepper
corn
cucumber
eggplant
garlic
ginger
grapes
jalepeno
kiwi
lemon
lettuce
mango
onion
orange
paprika
pear
peas
pineapple
pomegranate
potato
raddish
soy beans
spinach
sweetcorn
sweetpotato
tomato
turnip
watermelon


Делаем разделение 1 раз, выделим из train  - val и test

In [4]:
# Путь к папке с данными
data_folder = './train'

# Пути к папкам val и test
val_folder = './val/'
test_folder = './test/'

# Создание папок val и test, если они не существуют
os.makedirs(val_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Проход по всем папкам внутри папки с данными
for class_name in os.listdir(data_folder):
    class_path = os.path.join(data_folder, class_name)
    if os.path.isdir(class_path):
        # Получение списка изображений в текущей папке
        images = [f for f in os.listdir(class_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
        
        # Вычисление количества изображений для val и test (по 15% каждый)
        num_val = int(0.15 * len(images))
        num_test = int(0.15 * len(images))
        
        # Перемешивание изображений
        random.shuffle(images)
        
        # Разделение на val, test и перемещение изображений
        for i, image_name in enumerate(images):
            source_path = os.path.join(class_path, image_name)
            if i < num_val:
                destination_folder = os.path.join(val_folder, class_name)
            elif i < num_val + num_test:
                destination_folder = os.path.join(test_folder, class_name)
            else:
                break  # Остальные изображения оставляем в папке train

            os.makedirs(destination_folder, exist_ok=True)
            destination_path = os.path.join(destination_folder, image_name)
            shutil.move(source_path, destination_path)

print('Разделение изображений завершено.')


Разделение изображений завершено.


Считаем сколько изображений 

In [2]:
import os

def count_total_images(folder_path):
    total_count = 0
    for subdir in os.listdir(folder_path):
        if os.path.isdir(os.path.join(folder_path, subdir)):
            subdir_path = os.path.join(folder_path, subdir)
            count = sum([len(files) for _, _, files in os.walk(subdir_path)])
            #print(f'Количество изображений в {subdir}: {count}')
            total_count += count
    return total_count

# Путь к папке train
train_folder = './train'

# Подсчет общего количества изображений во всех подпапках
total_count_train = count_total_images(train_folder)
total_count_val = count_total_images('./val')
total_count_test = count_total_images('./test')

# Вывод общего количества изображений
print(f'Общее количество изображений в папке train: {total_count_train}')
print(f'Общее количество изображений в папке val: {total_count_val}')
print(f'Общее количество изображений в папке test: {total_count_test}')

Общее количество изображений в папке train: 2229
Общее количество изображений в папке val: 443
Общее количество изображений в папке test: 443


In [5]:
# Установите пути к папкам данных
train_folder = 'train'
val_folder = 'val'
test_folder = 'test'
# Задайте параметры для обработки изображений
batch_size = 32
target_size = (224, 224)  # Выберите желаемый размер изображений

# Создайте генераторы для данных обучения, валидации и тестирования
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')
val_generator = val_datagen.flow_from_directory(
    val_folder,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')

# Создайте и обучите модель
base_model = MobileNetV2(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
dropout_rate = 0.5
x = base_model.output
x = Dropout(dropout_rate)(x)  # Добавьте слой Dropout
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
# Установите желаемую скорость обучения (learning_rate)
custom_learning_rate = 0.0001

# Создайте оптимизатор с настраиваемой скоростью обучения
custom_adam_optimizer = Adam(learning_rate=custom_learning_rate)

model.compile(optimizer=custom_adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# Установите количество эпох и обучите модель
epochs = 20
history = model.fit(train_generator, validation_data=val_generator, epochs=epochs)
# Сохраните модель
model.save('my_classification_model.h5')

Found 2229 images belonging to 36 classes.
Found 443 images belonging to 36 classes.
Epoch 1/20
16/70 [=====>........................] - ETA: 1:13 - loss: 3.5143 - accuracy: 0.0703

c:\Users\user\anaconda3\envs\bot\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


70/70 [==============================] - 112s 2s/step - loss: 2.7016 - accuracy: 0.3477 - val_loss: 1.7893 - val_accuracy: 0.5847
Epoch 2/20
70/70 [==============================] - 104s 1s/step - loss: 1.3053 - accuracy: 0.6990 - val_loss: 1.1132 - val_accuracy: 0.7494
Epoch 3/20
70/70 [==============================] - 105s 2s/step - loss: 0.8918 - accuracy: 0.7663 - val_loss: 0.9412 - val_accuracy: 0.7472
Epoch 4/20
70/70 [==============================] - 100s 1s/step - loss: 0.7119 - accuracy: 0.8093 - val_loss: 0.8100 - val_accuracy: 0.7856
Epoch 5/20
70/70 [==============================] - 98s 1s/step - loss: 0.5863 - accuracy: 0.8457 - val_loss: 0.7606 - val_accuracy: 0.7788
Epoch 6/20
70/70 [==============================] - 103s 1s/step - loss: 0.5108 - accuracy: 0.8537 - val_loss: 0.7255 - val_accuracy: 0.8036
Epoch 7/20
70/70 [==============================] - 101s 1s/step - loss: 0.4512 - accuracy: 0.8748 - val_loss: 0.7112 - val_accuracy: 0.7946
Epoch 8/20
70/70 [=======

c:\Users\user\anaconda3\envs\bot\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# График точности (accuracy) на обучающем и валидационном наборах данных
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# График функции потерь (loss) на обучающем и валидационном наборах данных
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

 Делаем предсказание и выводим результат.

In [7]:
# Загрузка названий классов из файла
with open('French_labels copy.txt', 'r') as f:
    class_names = f.read().splitlines()
    
# Загрузка обученной модели
model = load_model('my_classification_model.h5')

# Загрузка изображения для предсказания
img = image.load_img('./морковь.jpg', target_size=(224, 224))

# Преобразование изображения в массив numpy и добавление дополнительной размерности
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Нормализация пикселей
img_array /= 255.

# Предсказание класса изображения
prediction = model.predict(img_array)

# Вывод предсказания
print('Предсказанный класс:', np.argmax(prediction))
print('Предсказанный класс:', class_names[np.argmax(prediction)])


1/1 [==============================] - 1s 664ms/step
Предсказанный класс: 6
Предсказанный класс: carrot


 Cоздаем и обучаем новую модель классификации на основе предварительно обученной модели VGG16, а затем сохраняем обученную модель для дальнейшего использования.

In [17]:
from tensorflow.keras.applications import VGG16

# Создайте и обучите модель
base_model = VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Установите количество эпох и обучите модель
epochs = 10
model.fit(train_generator, validation_data=val_generator, epochs=epochs)

# Сохраните модель
model.save('my_classification_model2.h5')

Epoch 1/10
70/70 [==============================] - 247s 4s/step - loss: 3.1927 - accuracy: 0.1615 - val_loss: 2.6391 - val_accuracy: 0.3160
Epoch 2/10
70/70 [==============================] - 243s 3s/step - loss: 2.3700 - accuracy: 0.3813 - val_loss: 2.1179 - val_accuracy: 0.4041
Epoch 3/10
70/70 [==============================] - 243s 3s/step - loss: 1.9430 - accuracy: 0.4868 - val_loss: 1.8767 - val_accuracy: 0.4740
Epoch 4/10
70/70 [==============================] - 245s 3s/step - loss: 1.7111 - accuracy: 0.5437 - val_loss: 1.7389 - val_accuracy: 0.4989
Epoch 5/10
70/70 [==============================] - 236s 3s/step - loss: 1.5129 - accuracy: 0.5868 - val_loss: 1.6866 - val_accuracy: 0.5102
Epoch 6/10
70/70 [==============================] - 233s 3s/step - loss: 1.4078 - accuracy: 0.6110 - val_loss: 1.5374 - val_accuracy: 0.5214
Epoch 7/10
70/70 [==============================] - 236s 3s/step - loss: 1.3084 - accuracy: 0.6330 - val_loss: 1.4657 - val_accuracy: 0.5576
Epoch 8/10
70

 Делаем предсказание и выводим результат.

In [9]:
# Загрузка названий классов из файла
with open('French_labels copy.txt', 'r') as f:
    class_names = f.read().splitlines()
    
# Загрузка обученной модели VGG16
model = load_model('my_classification_model2.h5')

# Загрузка изображения для предсказания
img = image.load_img('./банан.jpg', target_size=(224, 224))

# Преобразование изображения в массив numpy и добавление дополнительной размерности
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

# Нормализация пикселей
img_array /= 255.

# Предсказание класса изображения
prediction = model.predict(img_array)

# Вывод предсказания
print('Предсказанный класс:', class_names[np.argmax(prediction)])

1/1 [==============================] - 0s 224ms/step
Предсказанный класс: banana


Для увеличения точности применили custom_learning_rate = 0.0001, early_stopping, model_checkpoint

In [3]:
# Установите пути к папкам данных
train_folder = 'train'
val_folder = 'val'
test_folder = 'test'
# Задайте параметры для обработки изображений
batch_size = 32
target_size = (224, 224)  # Выберите желаемый размер изображений

# Создайте генераторы для данных обучения, валидации и тестирования
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')
val_generator = val_datagen.flow_from_directory(
    val_folder,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical')

# Создайте и обучите модель
base_model = MobileNetV2(input_shape=(224, 224, 3), weights='imagenet', include_top=False)
dropout_rate = 0.5
x = base_model.output
x = Dropout(dropout_rate)(x)  # Добавьте слой Dropout
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
# Установите желаемую скорость обучения (learning_rate)
custom_learning_rate = 0.0001

# Создайте оптимизатор с настраиваемой скоростью обучения
custom_adam_optimizer = Adam(learning_rate=custom_learning_rate)

model.compile(optimizer=custom_adam_optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
# Установите количество эпох и обучите модель
epochs = 30

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
# Настройка ранней остановки
early_stopping = EarlyStopping(
    monitor='val_loss',  # Мониторим функцию потерь на валидационном наборе данных
    patience=5,           # Количество эпох без улучшений перед остановкой
    restore_best_weights=True  # Восстановить веса модели до лучшей эпохи
)

# Настройка сохранения лучшей модели
model_checkpoint = ModelCheckpoint(
    'best_model.h5',  # Имя файла для сохранения лучшей модели
    monitor='val_loss',  # Мониторим функцию потерь на валидационном наборе данных
    save_best_only=True,  # Сохранять только лучшую модель
    mode='min',           # Режим мониторинга: 'min' для минимизации функции потерь
    verbose=1
)


history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs,
    callbacks=[early_stopping, model_checkpoint]  # Добавьте обратные вызовы
)



Found 2229 images belonging to 36 classes.
Found 443 images belonging to 36 classes.
Epoch 1/30
12/70 [====>.........................] - ETA: 1:08 - loss: 3.6182 - accuracy: 0.0483

c:\Users\user\anaconda3\envs\bot\lib\site-packages\PIL\Image.py:975: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


70/70 [==============================] - ETA: 0s - loss: 2.7320 - accuracy: 0.3544
Epoch 1: val_loss improved from inf to 1.79547, saving model to best_model.h5
70/70 [==============================] - 105s 1s/step - loss: 2.7320 - accuracy: 0.3544 - val_loss: 1.7955 - val_accuracy: 0.6027
Epoch 2/30


c:\Users\user\anaconda3\envs\bot\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


70/70 [==============================] - ETA: 0s - loss: 1.3235 - accuracy: 0.7061
Epoch 2: val_loss improved from 1.79547 to 1.10442, saving model to best_model.h5
70/70 [==============================] - 102s 1s/step - loss: 1.3235 - accuracy: 0.7061 - val_loss: 1.1044 - val_accuracy: 0.7088
Epoch 3/30
70/70 [==============================] - ETA: 0s - loss: 0.9022 - accuracy: 0.7712
Epoch 3: val_loss improved from 1.10442 to 0.90831, saving model to best_model.h5
70/70 [==============================] - 102s 1s/step - loss: 0.9022 - accuracy: 0.7712 - val_loss: 0.9083 - val_accuracy: 0.7517
Epoch 4/30
70/70 [==============================] - ETA: 0s - loss: 0.7029 - accuracy: 0.8179
Epoch 4: val_loss improved from 0.90831 to 0.80032, saving model to best_model.h5
70/70 [==============================] - 103s 1s/step - loss: 0.7029 - accuracy: 0.8179 - val_loss: 0.8003 - val_accuracy: 0.7720
Epoch 5/30
70/70 [==============================] - ETA: 0s - loss: 0.5963 - accuracy: 0.8318

In [ ]:
import matplotlib.pyplot as plt
# Получите данные о точности и потерях из объекта history
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Создайте графики
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(accuracy, label='Точность')
plt.plot(val_accuracy, label='Точность на валидации')
plt.legend()
plt.title('График точности')
plt.xlabel('Эпохи')
plt.ylabel('Точность')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Потери')
plt.plot(val_loss, label='Потери на валидации')
plt.legend()
plt.title('График потерь')
plt.xlabel('Эпохи')
plt.ylabel('Потери')

plt.tight_layout()
plt.show()
